# **Imports and Variables**

In [ ]:
!pip install cleverhans

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import app
from easydict import EasyDict
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization

from cleverhans.tf2.attacks.carlini_wagner_l2 import carlini_wagner_l2 as cw2
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method as fgsm
from cleverhans.tf2.attacks.basic_iterative_method import basic_iterative_method as bim
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent as pgd

In [ ]:
split = 1
def ld_mnist():
    """Load training and test data."""

    def convert_types(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label

    dataset, info = tfds.load(
        "mnist", data_dir="gs://tfds-data/datasets", with_info=True, as_supervised=True
    )
    mnist_train, mnist_test = dataset["train"], dataset["test"]
    mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(128)
    mnist_test = mnist_test.map(convert_types).batch(128)
    return EasyDict(train=mnist_train, test=mnist_test)

# **PGD**

In [ ]:
epochs = 20
eps = [0.05, 0.1, 0.15]

loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizers = [tf.optimizers.Adam(learning_rate=0.001),
              tf.optimizers.Adadelta(),
              tf.optimizers.Adagrad(),
              tf.optimizers.Adamax(),
              tf.optimizers.Nadam(),
              tf.optimizers.RMSprop(),
              tf.optimizers.SGD()
              ]
activations = ["relu"#,
               #"sigmoid",
               #"swish",
               #"tanh",
               #"elu",
]

In [ ]:
class Net(Model):
    def __init__(self, act="relu"):
        super(Net, self).__init__()
        self.conv1 = Conv2D(64, 8, strides=(2, 2), activation=act, padding="same")
        self.bc1 = BatchNormalization()
        self.conv2 = Conv2D(128, 6, strides=(2, 2), activation=act, padding="valid")
        self.bc2 = BatchNormalization()
        self.conv3 = Conv2D(128, 5, strides=(1, 1), activation=act, padding="valid")
        self.bc3 = BatchNormalization()
        self.dropout = Dropout(0.25)
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation=act)
        self.bc4 = BatchNormalization()
        self.dense2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        #x = self.bc1(x)
        x = self.conv2(x)
        #x = self.bc2(x)
        x = self.conv3(x)
        #x = self.bc3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.dense1(x)
        #x = self.bc4(x)
        return self.dense2(x)

optimizer = tf.optimizers.Adam(learning_rate=0.001)

def results(use_defense, epsilon, model):
#  for activation in activations:
      #for epsilon in eps:
          # Load training and test data
          data = ld_mnist()
          model = model

          # Metrics to track the different accuracies.
          train_loss = tf.metrics.Mean(name="train_loss")
          test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
          test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
          test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()

          @tf.function
          def train_step(x, y):
              with tf.GradientTape() as tape:
                  predictions = model(x)
                  loss = loss_object(y, predictions)
              gradients = tape.gradient(loss, model.trainable_variables)
              optimizer.apply_gradients(zip(gradients, model.trainable_variables))
              train_loss(loss)

          # Train model with adversarial training
          for epoch in range(epochs):
              # keras like display of progress
              progress_bar_train = tf.keras.utils.Progbar(60000)
              for (x, y) in data.train:
                  if use_defense:
                      # Replace clean example with adversarial example for adversarial training
                      x = pgd(model, x, epsilon, 0.01, 40, np.inf)
                  train_step(x, y)
                  progress_bar_train.add(x.shape[0], values=[("loss", train_loss.result())])

          # Evaluate on clean and adversarial data
          progress_bar_test = tf.keras.utils.Progbar(10000)
          for x, y in data.test:
              y_pred = model(x)
              test_acc_clean(y, y_pred)

              x_fgm = pgd(model, x, epsilon, 0.01, 40, np.inf)
              y_pred_fgm = model(x_fgm)
              test_acc_fgsm(y, y_pred_fgm)

              progress_bar_test.add(x.shape[0])
          if use_defense:
            string = "PGD Defense (%): {:.3f}"
          else:
            string = "PGD (%): {:.3f}"
          f = open("results.txt", "a")
          x = "epsilon: " + str(epsilon) + "\n" + "activation: " + str(activation) + " " + "\n" + "Clean (%): {:.3f}".format(test_acc_clean.result() * 100) + "%"
          print(x)
          f.write(x + "\n" + "\n")
          x = "epsilon: " + str(epsilon) + "\n" + "activation: " + str(activation) + " " + "\n" + string.format(test_acc_fgsm.result() * 100) + "%"
          f.write(x + "\n" + "\n")
          print(x)
          f.close()

In [ ]:
# No defense
eps = [0.1, 0.15, 0.3]
for activation in activations:
  for ep in eps:
    results(False, ep, Net(activation))

In [ ]:
# With defense
eps = [0.3]
for activation in activations:
  for ep in eps:
    results(True, ep, Net(activation))

10000/10000 [==============================] - 9s 946us/step
epsilon: 0.3
activation: relu 
Clean (%): 98.960%
epsilon: 0.3
activation: relu 
PGD Defense (%): 93.770%


# **FGSM**

In [ ]:
loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

activations = ["relu",
               #"sigmoid",
               #"elu",
               ]
eps = [0.1, 0.15, 0.3]#0.1, 0.15, 0.3]
epochs = 10

lr = 0.005#0.001

optimizer = tf.optimizers.Adam(learning_rate=lr)

In [ ]:
class Net1(Model):
    def __init__(self, act="relu"):
        super(Net1, self).__init__()
        self.conv1 = Conv2D(64, 8, strides=(2, 2), activation=act, padding="same")
        self.conv2 = Conv2D(128, 6, strides=(2, 2), activation=act, padding="valid")
        self.conv3 = Conv2D(128, 5, strides=(1, 1), activation=act, padding="valid")
        self.dropout = Dropout(0.25)
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation=act)
        self.dense2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [ ]:
def results(use_defense, epsilon):
  for activation in activations:
          # Load training and test data
          data = ld_mnist()
          model = Net1(activation)

          # Metrics to track the different accuracies.
          train_loss = tf.metrics.Mean(name="train_loss")
          test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
          test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
          test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()

          @tf.function
          def train_step(x, y):
              with tf.GradientTape() as tape:
                  predictions = model(x)
                  loss = loss_object(y, predictions)
              gradients = tape.gradient(loss, model.trainable_variables)
              optimizer.apply_gradients(zip(gradients, model.trainable_variables))
              train_loss(loss)

          # Train model with adversarial training
          for epoch in range(epochs):
              # keras like display of progress
              progress_bar_train = tf.keras.utils.Progbar(60000)
              for (x, y) in data.train:
                  if use_defense:
                      # Replace clean example with adversarial example for adversarial training
                      x = fgsm(model, x, epsilon, np.inf)
                  train_step(x, y)
                  progress_bar_train.add(x.shape[0], values=[("loss", train_loss.result())])

          # Evaluate on clean and adversarial data
          progress_bar_test = tf.keras.utils.Progbar(10000)
          for x, y in data.test:
              y_pred = model(x)
              test_acc_clean(y, y_pred)

              x_fgm = fgsm(model, x, epsilon, np.inf)
              y_pred_fgm = model(x_fgm)
              test_acc_fgsm(y, y_pred_fgm)

              progress_bar_test.add(x.shape[0])

          if use_defense:
            string = "FGSM Defense (%): {:.3f}"
          else:
            string = "FGSM (%): {:.3f}"
          
          #del model

          f = open("results.txt", "a")
          x = "epsilon: " + str(epsilon) + "\n" + "activation: " + str(activation) + " " + "\n" + "Clean (%): {:.3f}".format(test_acc_clean.result() * 100) + "%"
          print(x)
          f.write(x + "\n" + "\n")
          x = "epsilon: " + str(epsilon) + "\n" + "activation: " + str(activation) + " " + "\n" + string.format(test_acc_fgsm.result() * 100) + "%"
          f.write(x + "\n" + "\n")
          print(x)
          f.close()

In [ ]:
# No Defense
eps = [0.1, 0.15, 0.3]
for ep in eps:
  results(use_defense=False, epsilon=ep)

10000/10000 [==============================] - 3s 256us/step
epsilon: 0.1
activation: relu 
Clean (%): 98.430%
epsilon: 0.1
activation: relu 
FGSM (%): 66.810%
10000/10000 [==============================] - 3s 259us/step
epsilon: 0.15
activation: relu 
Clean (%): 11.350%
epsilon: 0.15
activation: relu 
FGSM (%): 11.350%
10000/10000 [==============================] - 3s 329us/step
epsilon: 0.3
activation: relu 
Clean (%): 11.350%
epsilon: 0.3
activation: relu 
FGSM (%): 11.350%


In [ ]:
# Defense
for ep in eps:
  results(True, ep)

20992/60000 [=========>....................] - ETA: 14s - loss: 1.5038

KeyboardInterrupt: ignored